# Titanic classification

In this document, we 

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Let's see the data

In [22]:
data_train = pd.read_csv('../input/titanic/train.csv')
target = 'Survived'
features = np.setdiff1d(data_train.columns.tolist(), target)

data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Some features seems irelevant and not usefull for prediction: `PassengerId`, `Name` and `Ticket`. We should remove them from the dataset.

In [24]:
drop_features = ['PassengerId', 'Name', 'Ticket']
data_train = data_train.drop(drop_features, axis=1)

In [25]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 62.7+ KB


As we can see there is lot of missing values for the features `Age` and `Cabin`. Before doing some ML, we must take care of the NaNs. 

In [10]:
import sys
sys.path.append('../script/')
from functions import get_catecorials, get_dummies

In [16]:
categorical_features = get_catecorials(data_train, target)
data_categorical = data_train[categorical_features]
data_categorical

,Cabin,Embarked,Name,Sex,Ticket
0,NaN,S,"Braund, Mr. Owen Harris",male,A/5 21171
1,C85,C,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599
2,NaN,S,"Heikkinen, Miss. Laina",female,STON/O2. 3101282
3,C123,S,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803
4,NaN,S,"Allen, Mr. William Henry",male,373450
5,NaN,Q,"Moran, Mr. James",male,330877
6,E46,S,"McCarthy, Mr. Timothy J",male,17463
7,NaN,S,"Palsson, Master. Gosta Leonard",male,349909
8,NaN,S,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742
9,NaN,C,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736
